In [1]:
import gc
import os
import glob
import zipfile
import warnings
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import cv2
import PIL
from PIL import ImageOps, ImageFilter, ImageDraw

In [2]:
DATA_PATH = './data/splitlabel/'
IMAGE_PATH = './data/467case_cut-20190916T015832Z-001/'
os.listdir(DATA_PATH)

['test_문-격자무늬 창.xlsx',
 'test_위치-집-아래.xlsx',
 'test_위치-집-정중앙.xlsx',
 'test_지면선-벽 밑면 있지만 지면 없음.xlsx',
 'test_지붕-그물무늬 음영.xlsx',
 'test_창문-격자무늬.xlsx',
 'test_창문-많다.xlsx',
 'test_창문-창문을 가린 커튼.xlsx',
 'train_문-격자무늬 창.xlsx',
 'train_위치-집-아래.xlsx',
 'train_위치-집-정중앙.xlsx',
 'train_지면선-벽 밑면 있지만 지면 없음.xlsx',
 'train_지붕-그물무늬 음영.xlsx',
 'train_창문-격자무늬.xlsx',
 'train_창문-많다.xlsx',
 'train_창문-창문을 가린 커튼.xlsx',
 '~$test_문-격자무늬 창.xlsx',
 '~$train_문-격자무늬 창.xlsx',
 '문-격자무늬 창.xlsx',
 '위치-집-아래.xlsx',
 '위치-집-정중앙.xlsx',
 '지면선-벽 밑면 있지만 지면 없음.xlsx',
 '지붕-그물무늬 음영.xlsx',
 '창문-격자무늬.xlsx',
 '창문-많다.xlsx',
 '창문-창문을 가린 커튼.xlsx']

In [3]:
house = ['문-격자무늬 창',
 '위치-집-아래',
 '위치-집-정중앙',
 '지면선-벽 밑면 있지만 지면 없음',
 '지붕-그물무늬 음영',
 '창문-격자무늬',
 '창문-많다',
 '창문-창문을 가린 커튼']

In [4]:
TRAIN_CROP_PATH = './train_crop'
TRAIN_IMG_PATH = os.path.join(IMAGE_PATH,'house')


df_train = []
df_test = []
for excel_name in house:
    df_train.append(pd.read_excel(os.path.join(DATA_PATH,'test_' + excel_name + '.xlsx')))
    df_train[house.index(excel_name)] = df_train[house.index(excel_name)].drop('Unnamed: 0', axis=1)
    df_train[house.index(excel_name)]['data'] = df_train[house.index(excel_name)].data.apply(lambda x: f'{x:0>4}')
    df_test.append(pd.read_excel(os.path.join(DATA_PATH,'test_' + excel_name + '.xlsx')))
    df_test[house.index(excel_name)] = df_train[house.index(excel_name)].drop('Unnamed: 0', axis=1)
    df_test[house.index(excel_name)]['data'] = df_train[house.index(excel_name)].data.apply(lambda x: f'{x:0>4}')
    
    

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
for i in range(len(house)):
    print(df_train[i].head(5))

In [5]:
tmp_imgs = df_train[0]['data'][100:105]
plt.figure(figsize=(12,20))

for num, f_name in enumerate(tmp_imgs):
    img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
    plt.subplot(5, 2, 2*num + 1)
    plt.title(f_name)
    plt.imshow(img)
    plt.axis('off')

<Figure size 864x1440 with 0 Axes>

In [6]:
from sklearn.model_selection import train_test_split

df_train[0][house[0]] = df_train[0][house[0]].astype('str')

df_train[0] = df_train[0][['data', house[0]]]
df_test[0] = df_test[0][['data']]

its = np.arange(df_train[0].shape[0])
train_idx, val_idx = train_test_split(its, train_size = 0.8, random_state=42)

X_train = df_train[0].iloc[train_idx, :]
X_val = df_train[0].iloc[val_idx, :]

print(X_train.shape)
print(X_val.shape)
print(df_train[0].shape)

(48, 2)
(12, 2)
(60, 2)


In [7]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [10]:
img_size = (224, 224)
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
nb_test_samples = len(df_train[0])
epochs = 20
batch_size = 32

# Define Generator config
train_datagen = ImageDataGenerator(
    horizontal_flip = True, 
    vertical_flip = False,
    preprocessing_function=preprocess_input
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Make Generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=X_train, 
    directory=TRAIN_IMG_PATH,
    x_col = 'data',
    y_col = house[0],
    target_size = img_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    seed=42
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=X_val, 
    directory=TRAIN_IMG_PATH,
    x_col='data',
    y_col=house[0],
    target_size=img_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test[0],
    directory=TRAIN_IMG_PATH,
    x_col='data',
    y_col=None,
    target_size= img_size,
    color_mode='rgb',
    class_mode=None,
    batch_size=batch_size,
    shuffle=False
)

Found 48 validated image filenames belonging to 2 classes.
Found 12 validated image filenames belonging to 1 classes.
Found 60 validated image filenames.


In [11]:
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D

In [12]:
mobileNetModel = MobileNet(weights='imagenet', include_top=False)

model = Sequential()
model.add(mobileNetModel)
model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax',kernel_initializer='he_normal'))
model.summary()

C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, None, None, 1024)  3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2050      
Total params: 3,230,914
Trainable params: 3,209,026
Non-trainable params: 21,888
_________________________________________________________________


In [13]:
from sklearn.metrics import f1_score

def micro_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='micro')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [14]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else : 
        return num_samples // batch_size

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

filepath = "my_mobile_model_{val_acc:.2f}_{val_loss:.4f}.h5"

ckpt = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')

callbackList = [ckpt]

history = model.fit_generator(
    train_generator,
    steps_per_epoch = get_steps(nb_train_samples, batch_size),
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = get_steps(nb_validation_samples, batch_size),
    callbacks = callbackList
)
gc.collect()

Epoch 1/20
1/2 [==============>...............] - ETA: 9s - loss: 1.3055 - acc: 0.1250

ValueError: You are passing a target array of shape (12, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model_list = sorted([i for i in os.listdir() if "my_" in i])
model_list

In [ ]:
model.load_weights(model_list[-1])

In [16]:
test_generator.reset()
prediction = model.predict_generator(
    generator = test_generator,
    steps = get_steps(nb_test_samples, batch_size),
    verbose=1
)

2/2 [==============================] - ETA:  - 2s 1s/step
